In [ ]:
import sys
from PyQt5 import QtCore, QtGui, QtWidgets, uic 
from PyQt5.QtWidgets import QMainWindow, QApplication, QDialog, QComboBox 
import pandas as pd 
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas 
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar 

import matplotlib.pyplot as plt 
import matplotlib


molwt = { 
        'O': 15.9994, 'Na': 22.99, 'Mg': 24.305, 'Al': 26.9815, 
        'Si': 28.086, 'P': 30.9738, 'K': 39.098, 'Ca': 40.078,
        'Ti': 47.867, 'V': 50.9415, 'Cr': 51.996, 'Mn': 54.938, 
        'Fe': 55.845, 'Ni': 58.693, 'Sr': 103.62 
}

elem = ('%SiO2', '%TiO2', '%Al2O3', '%FeO', '%Fe2O3', '%MnO', '%MgO', '%CaO', '%Na2O', '%K2O', '%P2O5', '%SrO', 
        'Si', 'Ti', 'Al', 'Fe', 'Mn', 'Cr', 'Sc', 'Co', 'Ca', 'Mg', 'K', 'P', 'Li', 'Yb', 'Lu', 
        'V', 'Cu', 'Ga', 'Rb', 'Zn', 'Sr', 'La', 'Y', 'Ba', 'Nd', 'Sm', 'Eu', 'Gd', 'Dy', 'Er', 'Pr', 'Ce', 'Pb')



molwts = pd.DataFrame(columns = ['value', 'el'])
molwts.loc['%SiO2'] = [molwt['Si']+molwt['O'], 'Si']
molwts.loc['%TiO2'] = [molwt['Ti']+molwt['O'], 'Ti']
molwts.loc['%Al2O3'] = [molwt['Al']*2+molwt['O'], 'Al']
molwts.loc['%SrO'] = [molwt['Sr']+molwt['O'], 'Sr']
molwts.loc['%FeO'] = [molwt['Fe']+molwt['O'], 'Fe']
molwts.loc['%Fe2O3'] = [molwt['Fe']*2+molwt['O']*3, 'Fe']
molwts.loc['%MnO'] = [molwt['Mn']+molwt['O'], 'Mn']
molwts.loc['%MgO'] = [molwt['Mg']+molwt['O'], 'Mg']
molwts.loc['%CaO'] = [molwt['Ca']+molwt['O'], 'Ca']
molwts.loc['%Na2O'] = [molwt['Na']*2+molwt['O'], 'Na']
molwts.loc['%K2O'] = [molwt['K']*2+molwt['O'], 'K']
molwts.loc['%P2O5'] = [molwt['P']*2+molwt['O']*5, 'P']


ppmwt = pd.DataFrame(columns = ['value', 'el'])

ppmwt.loc['%SiO2'] = [((molwts.loc['%SiO2'].value) * 10000)/(molwts.loc['%SiO2'].value), 'Si']
ppmwt.loc['%TiO2'] = [((molwts.loc['%TiO2'].value) * 10000)/(molwts.loc['%TiO2'].value), 'Ti']
ppmwt.loc['%Al2O3'] = [((molwts.loc['%Al2O3'].value) * 10000)/(molwts.loc['%Al2O3'].value), 'Al']
ppmwt.loc['%SrO'] = [((molwts.loc['%SrO'].value) * 10000)/(molwts.loc['%SrO'].value), 'Sr']
ppmwt.loc['%FeO'] = [((molwts.loc['%FeO'].value) * 10000)/(molwts.loc['%FeO'].value), 'Fe']
ppmwt.loc['%Fe2O3'] = [((molwts.loc['%Fe2O3'].value) * 10000)/(molwts.loc['%Fe2O3'].value), 'Fe']
ppmwt.loc['%MnO'] = [((molwts.loc['%MnO'].value) * 10000)/(molwts.loc['%MnO'].value), 'Mn']
ppmwt.loc['%MgO'] = [((molwts.loc['%MgO'].value) * 10000)/(molwts.loc['%MgO'].value), 'Mg']
ppmwt.loc['%CaO'] = [((molwts.loc['%CaO'].value) * 10000)/(molwts.loc['%CaO'].value), 'Ca']
ppmwt.loc['%Na2O'] = [((molwts.loc['%Na2O'].value) * 10000)/(molwts.loc['%Na2O'].value), 'Na']
ppmwt.loc['%K2O'] = [((molwts.loc['%K2O'].value) * 10000)/(molwts.loc['%K2O'].value), 'K']
ppmwt.loc['%P2O5'] = [((molwts.loc['%P2O5'].value) * 10000)/(molwts.loc['%P2O5'].value), 'P']

class Window(QtWidgets.QMainWindow): 
    def __init__(self, parent=None): 
        super(Window, self).__init__(parent) 
        uic.loadUi("profiler3.ui", self)

        self.standardload.clicked.connect(self.loadstnd)
        self.minload.clicked.connect(self.loadmin)
        self.confirm.clicked.connect(self.addelement)      
        self.calculate.clicked.connect(self.calculateci)
        self.pushButton.clicked.connect(self.saveproj)
        self.pushButton_2.clicked.connect(self.loadproj)

        self.figure = plt.figure() 
        self.canvas = FigureCanvas(self.figure) 
        self.toolbar = NavigationToolbar(self.canvas, self)
        self.pushButton_5.clicked.connect(self.plot) 
        self.plotLayout.addWidget(self.toolbar)
        self.plotLayout.addWidget(self.canvas)
        self.slidingwindow.addItems(['0', '5', '10', '20'])
        
        self.datatype.currentTextChanged.connect(self.datatypech)
        
        self.stval = {}
        self.filenames = pd.DataFrame()
        self.minmaxrange = pd.DataFrame()

    def loadstnd(self):
        fileName, _ = QtWidgets.QFileDialog.getOpenFileName(self, "Open File", "", "CSV Files (*.csv)")
        for n in range(10):
            try:   
                self.stand = pd.read_csv(fileName, sep=',', skiprows=n)
                self.stand.columns = self.stand.columns.str.extract(r'([a-zA-Z]+)', expand=False)
                if 'Si' in self.stand.columns:
                    break
            except:
                pass


        
        self.datatype.addItems(['CPS Standard'])

        self.el.addItems(elem)
        
        self.label_2.setText('Standard: '+fileName.split("/")[-1])
        
        self.filenames = self.filenames.append({'data': 'Standard: '+fileName.split("/")[-1]}, ignore_index=True)
        

    def loadmin(self):
        fileName, _ = QtWidgets.QFileDialog.getOpenFileName(self, "Open File", "", "CSV Files (*.csv)")
        for n in range(10):
            try:       
                self.miner = pd.read_csv(fileName, sep=',', skiprows=n)
                self.miner.columns = self.miner.columns.str.extract(r'([a-zA-Z]+)', expand=False)
                if 'Si' in self.miner.columns:
                    break 
            except:
                pass
     

        self.datatype.addItems(['CPS Mineral'])
        
        self.label_3.setText('Mineral: '+fileName.split("/")[-1])
        
        self.filenames = self.filenames.append({'data': 'Mineral: '+fileName.split("/")[-1]}, ignore_index=True)

    def addelement(self):
        e = self.el.currentText()
        if e.startswith('%'):
            self.stval[ppmwt.loc[e].el] = round((float(self.value.text()) * ppmwt.loc[e].value), 0)

        else: 
            self.stval[self.el.currentText()] = self.value.text()
            
        self.stvalstr = '\n'.join('{}: {} ppm'.format(key, val) for key, val in self.stval.items())
        self.standardlist.setPlainText(self.stvalstr)
        
        
    def calculateci(self):
        
        self.smin = int(self.minrange_2.text())
        self.smax = int(self.maxrange_2.text())
        
        self.mmin = int(self.minrange.text())
        self.mmax = int(self.maxrange.text())
        
        self.minmaxrange.s[0] = self.smin
        self.minmaxrange.s[1] = self.smax
        self.minmaxrange.m[0] = self.mmin
        self.minmaxrange.m[1] = self.mmax
        
        self.standbkg = self.stand[0:self.smin]
        self.standwobkg = self.stand[self.smin:self.smax]
        self.minerbkg = self.miner[0:self.mmin]
        
       
        
        self.ci = pd.DataFrame()
        self.elkeys = list(self.stval.keys())
        
        for k in self.elkeys:    
            try:
                e = '{}'.format(k)
                self.ci[e]=float(self.stval[e])*(self.miner[e]-self.minerbkg[e].mean())/(self.standwobkg[e].mean() - self.standbkg[e].mean())
            except:
                pass
        
            
            
            try:
                d = ppmwt[ppmwt.el == e]
                self.ci[d.index[0]] = self.ci[e] / d.value[0]
            except:
                pass
          
                
        

        
        self.cielements = list(self.ci)
        self.cipercent = [s for s in list(self.ci) if s.startswith('%')]
        self.cippm = [s for s in list(self.ci) if "%" not in s]
        self.ci['%sum'] = self.ci[self.cipercent].sum(axis=1)
        self.cipercent.append('%sum')
        
        for k in self.cipercent:
            self.ci['100'+k] = 100*self.ci[k]/self.ci['%sum']
        
        self.cinorm = [s for s in list(self.ci) if "100%" in s]
        
        
        
        
        self.cations = pd.DataFrame()
        
        for k in self.cinorm:
            try:
                e = '{}'.format(k)
                self.cations[molwts.loc[e[3:]].el] = self.ci[k] / molwts.loc[e[3:]].value
            except:
                pass
          
        
        self.cations['sum'] = self.cations.sum(axis=1)
        
        for k in list(self.cations):
            self.cations[k] = 5*self.cations[k]/self.cations['sum']
        
        self.cations = self.cations.drop(['sum'], 1)
        self.cations['sum'] = self.cations.sum(axis=1)
        
        self.plag = pd.DataFrame()
        self.plag['#Ca'] = 100*self.cations['Ca']/(self.cations['Ca']+self.cations['Na'])
        self.plag['M1'] = self.cations['Ca']+self.cations['Na']+self.cations['K']+self.cations['Sr']+self.cations['Mg']
        self.plag['An'] = 100 * self.cations['Ca'] / self.plag['M1']
        self.plag['Ab'] = 100 * self.cations['Na'] / self.plag['M1']
        self.plag['Or'] = 100 * self.cations['K'] / self.plag['M1']
        
        self.datatype.clear()
        self.datatype.addItems(['CPS Standard'])
        self.datatype.addItems(['CPS Mineral'])
        self.datatype.addItems(['Ci Mineral, ppm'])
        self.datatype.addItems(['Ci Mineral, %'])
        self.datatype.addItems(['Ci Mineral, 100%'])
        self.datatype.addItems(['Plagioclase'])

    def datatypech(self):
        if self.datatype.currentText() == 'CPS Standard':
            self.whattoshow.clear()
            self.whattoshow.addItems(list(self.stand)[1:])
        elif self.datatype.currentText() == 'CPS Mineral':
            self.whattoshow.clear()
            self.whattoshow.addItems(list(self.miner)[1:])
        elif self.datatype.currentText() == 'Ci Mineral, ppm':
            self.whattoshow.clear()
            self.whattoshow.addItems(self.cippm)
        elif self.datatype.currentText() == 'Ci Mineral, 100%':
            self.whattoshow.clear()
            self.whattoshow.addItems(self.cinorm)
        elif self.datatype.currentText() == 'Plagioclase':
            self.whattoshow.clear()
            self.whattoshow.addItems(list(self.plag))
        else:
            self.whattoshow.clear()
            self.whattoshow.addItems(self.cipercent)
            
            
    def saveproj(self):
        fileName, _ = QtWidgets.QFileDialog.getSaveFileName(self, "Save File", "", "Excel Files (*.xlsx)")
        
        writer = pd.ExcelWriter(fileName, engine='xlsxwriter')
        
        self.ci.to_excel(writer, sheet_name='ci', index=False)
        self.stand.to_excel(writer, sheet_name='standard', index=False)
        self.miner.to_excel(writer, sheet_name='mineral', index=False)
        stvaldf = pd.DataFrame([self.stval])
        stvaldf.to_excel(writer, sheet_name='StandardElValues', index=False)
        self.filenames.to_excel(writer, sheet_name='Datafile names', index=False)
        self.cations.to_excel(writer, sheet_name='cations', index=False)
        self.plag.to_excel(writer, sheet_name='Plagioclase', index=False)
        self.minmaxrange.to_excel(writer, sheet_name='minmaxrange', index=False)
        writer.save()
        
        
    def loadproj(self):
        fileName, _ = QtWidgets.QFileDialog.getOpenFileName(self, "Open File", "", "Excel Files (*.xlsx)")
        xls = pd.ExcelFile(fileName)
        self.ci = pd.read_excel(xls, 'ci')
        self.stand = pd.read_excel(xls, 'standard')
        self.miner = pd.read_excel(xls, 'mineral')
        self.stvaldf = pd.read_excel(xls, 'StandardElValues')
        self.filenames = pd.read_excel(xls, 'Datafile names')
        self.cations = pd.read_excel(xls, 'cations')
        self.plag = pd.read_excel(xls, 'Plagioclase')
        self.minmaxrange = pd.read_excel(xls, 'minmaxrange')
   
        self.stval = self.stvaldf.to_dict('records')[0]        
        
        self.stvalstr = '\n'.join('{}: {} ppm'.format(key, val) for key, val in self.stval.items())
        self.standardlist.setPlainText(self.stvalstr)
        
        self.datatype.clear()
        self.datatype.addItems(['CPS Standard'])
        self.datatype.addItems(['CPS Mineral'])
        self.datatype.addItems(['Ci Mineral, ppm'])
        self.datatype.addItems(['Ci Mineral, %'])
        self.datatype.addItems(['Ci Mineral, 100%'])
        self.datatype.addItems(['Plagioclase'])
        
        self.label_3.setText(self.filenames['data'][0])
        self.label_2.setText(self.filenames['data'][1])
        
        self.cipercent = [s for s in list(self.ci) if s.startswith('%')]
        self.cippm = [s for s in list(self.ci) if "%" not in s]
        self.cinorm = [s for s in list(self.ci) if "100%" in s]
        
        self.smin = int(self.minmaxrange.s[0])
        self.minrange_2.setText(str(self.smin))
        self.smax = int(self.minmaxrange.s[1])
        self.maxrange_2.setText(str(self.smax))
        
        self.mmin = int(self.minmaxrange.m[0])
        self.minrange.setText(str(self.mmin))
        self.mmax = int(self.minmaxrange.m[1])
        self.maxrange.setText(str(self.mmax))      
        
        
        self.standbkg = self.stand[0:self.smin]
        self.standwobkg = self.stand[self.smin:self.smax]
        self.minerbkg = self.miner[0:self.mmin]
        
        
        self.el.addItems(elem)

    def plot(self):
        
        e = self.whattoshow.currentText()
        
        if self.datatype.currentText() == 'CPS Standard':
            d = self.stand
        elif self.datatype.currentText() == 'CPS Mineral':
            d = self.miner
        elif self.datatype.currentText() == 'Plagioclase':
            d = self.plag
            d = d[self.mmin:self.mmax]
        else:
            d = self.ci
            d = d[self.mmin:self.mmax]

        self.figure.clear()
        ax = self.figure.add_subplot(111) 
        
        if e.startswith('%'):
            ax.set_ylabel(e[1:]+', %')         
        elif e.startswith('100%'):
            ax.set_ylabel(e[4:]+', % normalized to 100')
        elif self.datatype.currentText() == 'Plagioclase':
            ax.set_ylabel(e)
        elif self.datatype.currentText() == 'CPS Standard':
            ax.set_ylabel(e+', CPS')
        elif self.datatype.currentText() == 'CPS Mineral':   
            ax.set_ylabel(e+', CPS')
        else:
            ax.set_ylabel(e+', ppm')
        
        if self.slidingwindow.currentText() == '5':
            y = d[e].rolling(5).mean()
        elif self.slidingwindow.currentText() == '10':
            y = d[e].rolling(10).mean()
        elif self.slidingwindow.currentText() == '20':
            y = d[e].rolling(20).mean()   
        else:
            y = d[e]
       


        ax.plot(y, 'b-') 
        
        ax.set_xlabel('Measurements')
        

            
        self.figure.tight_layout()

        self.canvas.draw()      
 
if __name__ == '__main__':

        import sys
        app = QtWidgets.QApplication(sys.argv)

        window = Window()
        window.show()
        sys.exit(app.exec_())

In [3]:
exit